<a href="https://colab.research.google.com/github/AgustinRGomez/EstacionamientoUnahur/blob/main/Python_Estacionamiento_UNAHUR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Recordatorio de como funcionaba una lista.***


In [8]:
lista = []
lista.append(("AAA345",12,2))
lista.append(("GDT111",3,4))
lista.append(("AFA313",5,6))
lista.append(("AFGA340",7,8))
lista.append(("AVC491",10,0))
print(lista[0][0]) #imprimir contenido de la posicion 0.
print(lista[0][1]) #imprimir contenido de la posicion 1.
print(lista[0][2]) #imprimir contenido de la posicion 2.

AAA345
12
2


## ***Armado de la Clase Estacionamiento y de la Clase Sector.***

In [10]:
#clase TDAS
import numpy as np
class Estacionamiento:
  def __init__(self):
    self.sector_general = Sector(100) #declaro 3 sectores : general,docente y alumno con sus respectivos tamaños.
    self.sector_docente = Sector(50)
    self.sector_alumno = Sector(50)
    self.recaudacion = 0

  def add_matricula(self,matricula,hora_ent,sector):
      #Ademas de verificar el sector en el que quiero alojar el vehiculo,verifico si dicho vehiculo esta estacionado en otro sector.
      if type(sector) == str:
        if ((sector == "docente") and not(self.sector_general.validate_car(matricula)) and not(self.sector_alumno.validate_car(matricula)) ):
             self.sector_docente.add_car(matricula,hora_ent)

        elif (sector == "alumno")and not(self.sector_docente.validate_car(matricula)) and not(self.sector_general.validate_car(matricula)) :
            self.sector_alumno.add_car(matricula,hora_ent)

        elif (sector == "general" )and not(self.sector_docente.validate_car(matricula)) and not(self.sector_alumno.validate_car(matricula)):
            self.sector_general.add_car(matricula,hora_ent)
        else : raise Exception (f"El vehiculo no se puede agregar porque esta estacionado en otro Sector.")

      else: raise Exception("El sector de estacionamiento tiene que ser de tipo str")

  def delete_matricula(self,matricula,hora_sal):
      #Verifico donde esta el vehiculo ,si lo encuentra lo elimina de dicho sector.
      if  self.sector_docente.validate_car(matricula) :
        self.sector_docente.delete_car(matricula,hora_sal)

      elif self.sector_alumno.validate_car(matricula) :
        self.sector_alumno.delete_car(matricula,hora_sal)

      elif self.sector_general.validate_car(matricula) :
        self.sector_general.delete_car(matricula,hora_sal)
      else :
        raise Exception("El vehiculo que quiere eliminar no se encuentra en ningun sector del estacionamiento.")

  def getRecaudacion(self):

      self.recaudacion = self.sector_docente.dif_time * 10 + self.sector_alumno.dif_time * 5 + self.sector_general.dif_time * 20
      print(f"El total a recaudar es {self.sector_docente.dif_time} * $10 + {self.sector_alumno.dif_time} * $5 + {self.sector_general.dif_time} * $20  = {self.recaudacion}")

  #Obtengo la recaudacion que acumula el estacionamiento.
  # Para el metodo getRecaudacion llamo desde los sectores la diferencia de tiempo que acumulo en cada sector.

  def IsMatriculaInEstacionamientoAt(self,matricula,hora_det):
    #recorro en los registros si esta la matricula a consultar.
    if self.sector_docente.isCarInSectorAt(matricula,hora_det):
      print(f"El vehiculo con la matricula {matricula} si estuvo a la {hora_det} horas en el estacionamiento.")
    elif self.sector_alumno.isCarInSectorAt(matricula,hora_det):
      print(f"El vehiculo con la matricula {matricula} si estuvo a la {hora_det} horas en el estacionamiento.")
    elif self.sector_general.isCarInSectorAt(matricula,hora_det):
      print(f"El vehiculo con la matricula {matricula} si estuvo a la {hora_det} horas en el estacionamiento.")
    else:
      raise Exception(f"El vehiculo con la matricula {matricula} no estuvo a la {hora_det} horas en el estacionamiento.")

class Sector:
  def __init__(self,tamaño):
    self.tamaño = tamaño # Capacidad que tiene el sector de almacenar vehiculos.
    self.lista_car = []  # Lista donde se almacenara los vehiculos.
    self.cant_v = 0      # Cantidad de vehiculos en el sector.
    self.dif_time = 0    # El tiempo que estuvo el vehiculo estacionado.Desde que entro hasta que salio.
    self.registro = []   # Lista de vehiculos registrados en el sector.

  def add_car(self,matricula,hora_ent):
    if (not self.validate_car(matricula)):                      #Verifica si la matricula ya esta en la lista.
        if (type(matricula) == str and len(matricula) == 6):    #Chequea el tipo de la variable matricula y su tamaño igual a 6.
            if hora_ent >= 0 :                                  #Consulto si la hora_ent es mayor o igual a 0.
               if self.cant_v < self.tamaño:                    #Consulto si la cantidad de vehiculos supera el tamaño que tiene el Sector.
                  hora_sal = 0
                  self.lista_car.append([matricula,hora_ent,hora_sal])   #Agrego matricula y la hora de entrada a la lista del sector.
                  self.cant_v+= 1                               #Aumenta la cantidad de vehiculos que tiene la lista del sector.

                  print(f"El vehiculo {matricula} ingreso a las {hora_ent} horas")

               else: raise Exception("El sector esta ocupado por lo tanto no puedo almacenar vehiculos.")
            else: raise Exception("La hora de entrada no puede ser negativa.")
        else: raise Exception("La matricula tiene que ser str con una longitud de 6.")
    else: raise Exception("No se puede agregar un vehiculo que esta dentro del estacionamiento.")

  def delete_car(self,matricula,hora_sal):
    for i in range(len(self.lista_car)):                             #Recorro la lista de vehiculos.
       if self.lista_car[i][0] == matricula:                         #Verifico que el vehiculo esta en la lista.
          self.lista_car[i][2] = hora_sal
          self.dif_time += self.lista_car[i][2] - self.lista_car[i][1]#Obtengo el tiempo que estuvo el vehiculo estacionado.

          self.registro.append(self.lista_car[i])                    #Registro el vehiculo con los datos que tiene antes de que salga.
          del self.lista_car[i]                                      #Elimina el vehiculo de la lista.
          self.cant_v -= 1                                           #Indica que se desocupo un lugar en la lista.
          print(f"El vehiculo {matricula} se retiro a las {hora_sal} horas")
          break

  def validate_car(self,matricula):
     for i in range(len(self.lista_car)):                 #Recorro la lista de vehiculos.
          if matricula == self.lista_car[i][0]:           #Verifico que la matricula del vehiculo este en la lista.
             return True                                  #Si esta dicho vehiculo en la lista retorna un True.
             break

  def isCarInSectorAt(self,matricula,hora_det):
     if hora_det >= 0 :                                            #Chequeo que la hora determinada sea mayor a 0 .
       if type(matricula) == str and len(matricula) == 6:          #Chequeo la matricula sea de tipo str con tamaño igual a 6.
        for i in range(len(self.registro)):
          matri = self.registro[i][0]                              #asigno matricula,hora de entrada y hora de salida a 3 variables.
          hora_ent = self.registro[i][1]
          hora_sal = self.registro[i][2]
          if (matri == matricula )and (hora_ent <= hora_det)  and (hora_det <= hora_sal or hora_sal == 0): #Verifico que la hora determinada este entre la hora de entrada y la de salida.
            return True                                                                   #Si se cumple la condicion retorna un True.
            break

       else: raise Exception("La matricula tiene que ser str con una longitud de 6.")
     else: raise Exception("La hora de entrada no puede ser negativa.")


## ***Verificacion de las Excepciones que tiene la clase Estacionamiento y la clase Sector***

In [11]:
e2 = Estacionamiento()
e2.add_matricula("AAA111",11,"docente")
e2.add_matricula("AAA111",11,"alumno")
e2.add_matricula("AAA111",11,"general")

El vehiculo AAA111 ingreso a las 11 horas


Exception: El vehiculo no se puede agregar porque esta estacionado en otro Sector.

In [12]:
e4 = Estacionamiento()
e4.delete_matricula("AAD456",13)

Exception: El vehiculo que quiere eliminar no se encuentra en ningun sector del estacionamiento.

In [13]:
e5 = Estacionamiento()
e5.add_matricula("AAA111",11,"docente")
e5.add_matricula("BBB111",12,"alumno")
e5.add_matricula("CCC111",10,"general")
e5.delete_matricula("AAA111",13)
e5.delete_matricula("BBB111",15)
e5.delete_matricula("CCC111",13)
e5.getRecaudacion()
e5.IsMatriculaInEstacionamientoAt("AAA111",13)
e5.IsMatriculaInEstacionamientoAt("BBB111",14)
e5.IsMatriculaInEstacionamientoAt("CCC111",20)

El vehiculo AAA111 ingreso a las 11 horas
El vehiculo BBB111 ingreso a las 12 horas
El vehiculo CCC111 ingreso a las 10 horas
El vehiculo AAA111 se retiro a las 13 horas
El vehiculo BBB111 se retiro a las 15 horas
El vehiculo CCC111 se retiro a las 13 horas
El total a recaudar es 2 * $10 + 3 * $5 + 3 * $20  = 95
El vehiculo con la matricula AAA111 si estuvo a la 13 horas en el estacionamiento.
El vehiculo con la matricula BBB111 si estuvo a la 14 horas en el estacionamiento.


Exception: El vehiculo con la matricula CCC111 no estuvo a la 20 horas en el estacionamiento.

## ***Captura de una Exception de la clase Estacionamiento***

In [14]:
try:
  e3 = Estacionamiento()
  e3.add_matricula("AAA111",1,"docente")
  e3.add_matricula("AAA111",4,"docente")
except Exception as ex:
  assert str(ex) == "No se puede agregar un vehiculo que esta dentro del estacionamiento."

El vehiculo AAA111 ingreso a las 1 horas



## ***Funcionamiento correcto de la Clase Estacionamiento***

In [15]:
e1 = Estacionamiento()
e1.add_matricula("AAA111",11,"docente")
e1.add_matricula("BBB111",12,"alumno")
e1.add_matricula("CCC111",10,"general")

e1.delete_matricula("AAA111",13)
e1.delete_matricula("BBB111",15)
e1.delete_matricula("CCC111",13)

e1.getRecaudacion()

e1.add_matricula("ABC246",11,"alumno")
e1.add_matricula("BDE789",12,"alumno")
e1.add_matricula("FGH321",2,"general")

e1.delete_matricula("ABC246",13)
e1.delete_matricula("BDE789",15)
e1.delete_matricula("FGH321",5)

e1.getRecaudacion()
e1.IsMatriculaInEstacionamientoAt("ABC246",13)
e1.IsMatriculaInEstacionamientoAt("FGH321",4)

El vehiculo AAA111 ingreso a las 11 horas
El vehiculo BBB111 ingreso a las 12 horas
El vehiculo CCC111 ingreso a las 10 horas
El vehiculo AAA111 se retiro a las 13 horas
El vehiculo BBB111 se retiro a las 15 horas
El vehiculo CCC111 se retiro a las 13 horas
El total a recaudar es 2 * $10 + 3 * $5 + 3 * $20  = 95
El vehiculo ABC246 ingreso a las 11 horas
El vehiculo BDE789 ingreso a las 12 horas
El vehiculo FGH321 ingreso a las 2 horas
El vehiculo ABC246 se retiro a las 13 horas
El vehiculo BDE789 se retiro a las 15 horas
El vehiculo FGH321 se retiro a las 5 horas
El total a recaudar es 2 * $10 + 8 * $5 + 6 * $20  = 180
El vehiculo con la matricula ABC246 si estuvo a la 13 horas en el estacionamiento.
El vehiculo con la matricula FGH321 si estuvo a la 4 horas en el estacionamiento.



## ***Funcionamiento correcto de la Clase Sector***

In [16]:
s1 = Sector(5)
s1.add_car("AAA345",4)
s1.add_car("AVC491",1)
s1.add_car("AFG340",7)
s1.add_car("AFA313",5)
s1.add_car("GDT111",3)
print("lista original")
print(s1.lista_car)
print("lista original borrando AFA313")
s1.delete_car("AFA313",7)
print(s1.registro)
print(s1.lista_car)
s1.add_car("AFA313",3)
print("lista original agregado AFA313")
print(s1.lista_car)

s1.validate_car("AFA313")

s1.isCarInSectorAt("AFA313",6) #Verifica si el vehiculo esta en el estacionamiento sin que el vehiculo haya salido.

El vehiculo AAA345 ingreso a las 4 horas
El vehiculo AVC491 ingreso a las 1 horas
El vehiculo AFG340 ingreso a las 7 horas
El vehiculo AFA313 ingreso a las 5 horas
El vehiculo GDT111 ingreso a las 3 horas
lista original
[['AAA345', 4, 0], ['AVC491', 1, 0], ['AFG340', 7, 0], ['AFA313', 5, 0], ['GDT111', 3, 0]]
lista original borrando AFA313
El vehiculo AFA313 se retiro a las 7 horas
[['AFA313', 5, 7]]
[['AAA345', 4, 0], ['AVC491', 1, 0], ['AFG340', 7, 0], ['GDT111', 3, 0]]
El vehiculo AFA313 ingreso a las 3 horas
lista original agregado AFA313
[['AAA345', 4, 0], ['AVC491', 1, 0], ['AFG340', 7, 0], ['GDT111', 3, 0], ['AFA313', 3, 0]]


True